# Cold Publisher

In [25]:
USE {
    repositories {
        // Any additional repositories. Maven central is already included
        // maven("<url>")
    }
    dependencies {
        // Here we add kandy plotting library
//        implementation("org.slf4j:slf4j-simple:1.7.25")
        implementation("io.github.microutils:kotlin-logging:1.12.5")
    }
    } // see https://datalore.jetbrains.com/report/static/sBgNeHxfaKG1tVRDWl3KqU/DMtWM05XWOT2Oi6yxZjVkJ

The problem is found in one of the loaded libraries: check library imports, dependencies and repositories
Error compiling code:
@file:DependsOn("io.github.microutils:kotlin-logging:1.12.5")


Errors:
Unhandled exception during resolve

org.jetbrains.kotlinx.jupyter.exceptions.ReplLibraryException: The problem is found in one of the loaded libraries: check library imports, dependencies and repositories
	at org.jetbrains.kotlinx.jupyter.exceptions.ReplLibraryExceptionKt.rethrowAsLibraryException(ReplLibraryException.kt:32)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$ExecutionContext.doAddLibraries(CellExecutorImpl.kt:147)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$ExecutionContext.addLibraries(CellExecutorImpl.kt:182)
	at org.jetbrains.kotlinx.jupyter.api.KotlinKernelHost$DefaultImpls.addLibrary(KotlinKernelHost.kt:42)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$ExecutionContext.addLibrary(CellExecutorImpl.kt:126)
	at jupyter.kotlin.Cel

In [26]:
System.setProperties(Properties().apply { setProperty("logger.level.root", "info") })

In [27]:
System.getProperties()

{idea.io.use.nio2=true, logger.level.root=info}

In [14]:
val logger = mu.KotlinLogging.logger {  }

In [30]:
fun logging(message: String) {
    println("[${Thread.currentThread().name}] $message")
}

In [6]:
import java.util.concurrent.ExecutorService
import java.util.concurrent.Executors
import java.util.concurrent.Flow
import java.util.concurrent.Flow.Subscriber


class SimpleColdSubscription(private val iterator: Iterator<Int>, private val subscriber: Subscriber<in Int>) :
    Flow.Subscription {
    private val executorService = Executors.newSingleThreadExecutor()
    override fun request(n: Long) {
        executorService.submit {
            for (i in (0 until n)) {
                if (iterator.hasNext()) {
                    subscriber.onNext(iterator.next())
                } else {
                    subscriber.onComplete()
                    executorService.shutdown()
                    break
                }
            }

         }
    }

    override fun cancel() {
        subscriber.onComplete()
    }

}

In [9]:
import java.util.*

class SimpleColdPublisher : Flow.Publisher<Int> {
    override fun subscribe(subscriber: Flow.Subscriber<in Int>) {
        val iter = Collections.synchronizedList((1..10).toList()).iterator()
        val subscription = Line_6_jupyter.SimpleColdSubscription(iter, subscriber)
        subscriber.onSubscribe(subscription)

    }
}

In [31]:
class SimpleNamedSubscriber<T>(private val name: String) : Flow.Subscriber<T> {
    var subscription: Flow.Subscription? = null // 리모컨 역할을 함

    override fun onSubscribe(subscription: Flow.Subscription) {
        this.subscription = subscription
        this.subscription?.request(1)
        logging("onSubscribe")
    }

    override fun onError(throwable: Throwable) {
        logging("onError: $throwable")
    }

    override fun onComplete() {
        logging("onComplete")
    }

    override fun onNext(item: T) {
        logging("name: $name, onNext: $item")
        this.subscription?.request(1)
    }

    fun cancel() {
        logging("cancel")
        this.subscription?.cancel()
    }

}

In [32]:

val publisher = SimpleColdPublisher()

val subscriber1 = SimpleNamedSubscriber<Int>("subscriber1")
publisher.subscribe(subscriber1)

Thread.sleep(1000)

val subscriber2 = SimpleNamedSubscriber<Int>("subscriber2")
publisher.subscribe(subscriber2)


[Execution of code '...'] onSubscribe
[pool-5-thread-1] name: subscriber1, onNext: 1
[pool-5-thread-1] name: subscriber1, onNext: 2
[pool-5-thread-1] name: subscriber1, onNext: 3
[pool-5-thread-1] name: subscriber1, onNext: 4
[pool-5-thread-1] name: subscriber1, onNext: 5
[pool-5-thread-1] name: subscriber1, onNext: 6
[pool-5-thread-1] name: subscriber1, onNext: 7
[pool-5-thread-1] name: subscriber1, onNext: 8
[pool-5-thread-1] name: subscriber1, onNext: 9
[pool-5-thread-1] name: subscriber1, onNext: 10
[pool-5-thread-1] onComplete
[Execution of code '...'] onSubscribe
[pool-6-thread-1] name: subscriber2, onNext: 1
[pool-6-thread-1] name: subscriber2, onNext: 2
[pool-6-thread-1] name: subscriber2, onNext: 3
[pool-6-thread-1] name: subscriber2, onNext: 4
[pool-6-thread-1] name: subscriber2, onNext: 5
[pool-6-thread-1] name: subscriber2, onNext: 6
[pool-6-thread-1] name: subscriber2, onNext: 7
[pool-6-thread-1] name: subscriber2, onNext: 8
[pool-6-thread-1] name: subscriber2, onNext: 9
[